In [1]:
!pip install transformers datasets peft bitsandbytes torch accelerate sentencepiece pandas psutil



In [2]:
import torch
import pandas as pd
import numpy as np
import psutil
import glob
import os
import re
from datasets import Dataset
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM, TrainingArguments, Trainer, 
    DataCollatorForSeq2Seq, BitsAndBytesConfig
)
from peft import get_peft_model, LoraConfig, TaskType



<b>Функция для вычисления оптимального размера чанка <b/>

In [3]:
def get_optimal_chunk_size(min_chunk_size=10000, max_chunk_size=50000):
    """
    Рассчитывает оптимальный размер чанка на основе доступной RAM и VRAM.
    """
    total_ram = psutil.virtual_memory().available / (1024 ** 3)  # Доступная RAM
    total_vram = torch.cuda.get_device_properties(0).total_memory / (1024 ** 3) if torch.cuda.is_available() else 0  # VRAM
    estimated_chunk_size = int((total_ram + total_vram) * 2500)
    return max(min(estimated_chunk_size, max_chunk_size), min_chunk_size)



<b>Загрузка датасета и его обработка<b/>

In [4]:
df = pd.read_csv("output_final_4persent.csv", sep=";").dropna()  # Загружаем и очищаем датасет

if "text_wich_errors" in df.columns:
    df.rename(columns={"text_wich_errors": "text_with_errors"}, inplace=True)  # Исправляем название столбца

chunk_size = get_optimal_chunk_size()

# Проверяем, есть ли уже чанки, чтобы не создавать их повторно
if not any(fname.startswith("chunk_") for fname in os.listdir(".")):
    print("🔹 Чанки не найдены, создаем заново...")
    for i, start in enumerate(range(0, len(df), chunk_size)):
        df.iloc[start:start + chunk_size].to_csv(f"chunk_{i + 1}.csv", sep=";", index=False)
    print(f"✅ Датасет разбит на {i + 1} чанков.")
else:
    print("✅ Чанки уже существуют, повторное создание не требуется.")


✅ Чанки уже существуют, повторное создание не требуется.


<b>Загрузка модели и токенизатора<b/>

In [5]:
model_name = "UrukHan/t5-russian-spell"
tokenizer = AutoTokenizer.from_pretrained(model_name)

quantization_config = BitsAndBytesConfig(load_in_8bit=True)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, quantization_config=quantization_config, device_map="auto")
model.config.use_cache = False  # Отключаем кеширование


<b>Функция для токенизации данных<b/>

In [6]:
def preprocess_function(examples):
    """
    Токенизирует входные и целевые тексты, создавая labels для обучения.
    """
    model_inputs = tokenizer(examples["input_text"], padding="max_length", truncation=True, max_length=512)
    labels = tokenizer(examples["target_text"], padding="max_length", truncation=True, max_length=512)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)



<b>Настройка LoRA и обновление модели<b/>

In [7]:
lora_config = LoraConfig(task_type=TaskType.SEQ_2_SEQ_LM, r=8, lora_alpha=32, lora_dropout=0.1)
model = get_peft_model(model, lora_config)  # Добавляем LoRA

# Включаем градиенты только для LoRA-адаптеров
for name, param in model.named_parameters():
    if "lora" in name:
        param.requires_grad = True
    else:
        param.requires_grad = False

<b>Настройка параметров обучения<b/>

In [8]:
training_args = TrainingArguments(
    output_dir="./t5-spell-corrector",
    eval_strategy="steps",
    save_strategy="steps",
    save_steps=100,
    logging_steps=100,
    learning_rate=5e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=16,
    weight_decay=0.01,
    bf16=torch.cuda.is_bf16_supported(),
    optim="adamw_bnb_8bit",
    label_names=["labels"],
)


<b>Поиск сохраненных чекпоинтов<b/>


In [13]:
# Указываем директорию для чекпоинтов
checkpoint_dir = "./t5-spell-corrector"

import json

# Функция: Найти последний обработанный чанк
def find_last_processed_chunk():
    chunk_numbers = []
    for folder in glob.glob(f"{checkpoint_dir}/checkpoint-*"):
        match = re.search(r'checkpoint-(\d+)', folder)
        if match:
            step_number = int(match.group(1))  # Получаем номер чекпоинта
            chunk_number = min(step_number // 100 + 1, 5)  # Ограничиваем до 5 чанков
            chunk_numbers.append(chunk_number)
    return max(chunk_numbers) if chunk_numbers else 0

# Функция: Найти последний валидный чекпоинт
def find_last_valid_checkpoint():
    checkpoint_list = sorted(
        glob.glob(f"{checkpoint_dir}/checkpoint-*"),
        key=lambda x: int(re.search(r'checkpoint-(\d+)', x).group(1)) if re.search(r'checkpoint-(\d+)', x) else 0
    )
    for checkpoint in reversed(checkpoint_list):
        trainer_state_path = os.path.join(checkpoint, "trainer_state.json")
        if os.path.exists(trainer_state_path):
            return checkpoint  
    return None  

# Функция: Проверить, завершились ли все эпохи
def check_if_training_completed(last_checkpoint):
    trainer_state_path = os.path.join(last_checkpoint, "trainer_state.json")
    
    if os.path.exists(trainer_state_path):
        with open(trainer_state_path, "r") as f:
            state = json.load(f)
            current_epoch = state.get("epoch", None)
            total_epochs = training_args.num_train_epochs

            if current_epoch is not None and current_epoch >= total_epochs:
                return True  # Обучение завершено
    return False  # Нужно продолжить обучение

# 🔹 Определяем последний обработанный чанк и чекпоинт
last_processed_chunk = find_last_processed_chunk()
last_checkpoint = find_last_valid_checkpoint()

# 🔹 Проверяем, завершилось ли обучение
training_completed = check_if_training_completed(last_checkpoint)

print(f"📌 Последний обработанный чанк: {last_processed_chunk}")
if last_checkpoint:
    print(f"✅ Обнаружен последний рабочий чекпоинт: {last_checkpoint}")
    if training_completed:
        print("🎉 Обучение полностью завершено! 🚀")
    else:
        print("🔄 Обучение продолжается с текущей эпохи...")
else:
    print("❌ Рабочие чекпоинты не найдены, обучение начнется с нуля.")


📌 Последний обработанный чанк: 5
✅ Обнаружен последний рабочий чекпоинт: ./t5-spell-corrector/checkpoint-8436
🔄 Обучение продолжается с текущей эпохи...


<b>Запуск обучения<b/>

In [14]:
chunk_files = sorted(glob.glob("chunk_*.csv"))

if not chunk_files:
    raise FileNotFoundError("❌ Ошибка: Чанки не найдены! Убедись, что файлы `chunk_*.csv` существуют.")

last_processed_chunk = int(re.findall(r'\d+', last_checkpoint)[-1]) if last_checkpoint else 0

for i, chunk_file in enumerate(chunk_files):
    current_chunk_number = i + 1  

    if current_chunk_number <= last_processed_chunk:
        print(f"⏭️ Пропускаем чанк {current_chunk_number}, он уже обработан.")
        continue

    print(f"▶️ Обучение на чанке {current_chunk_number}/{len(chunk_files)}: {chunk_file}")

    df_chunk = pd.read_csv(chunk_file, sep=";")
    dataset = Dataset.from_pandas(df_chunk).train_test_split(test_size=0.1, seed=42)

    dataset = dataset.map(lambda x: {"input_text": "Исправь текст: " + x["text_with_errors"], "target_text": x["corrected_text"]})
    dataset = dataset.map(preprocess_function, batched=True)

    trainer = Trainer(model=model, args=training_args, train_dataset=dataset["train"], eval_dataset=dataset["test"], data_collator=data_collator)

    if last_checkpoint and os.path.exists(os.path.join(last_checkpoint, "trainer_state.json")):
        print(f"🔄 Продолжаем обучение с последнего рабочего чекпоинта: {last_checkpoint}")
        trainer.train(resume_from_checkpoint=last_checkpoint)
    else:
        print("🚀 Начинаем обучение с нуля")
        trainer.train()

    new_checkpoint = f"{checkpoint_dir}/checkpoint-{current_chunk_number}"
    trainer.save_model(new_checkpoint)
    last_checkpoint = new_checkpoint
    print(f"✅ Сохранён новый чекпоинт: {new_checkpoint}")


⏭️ Пропускаем чанк 1, он уже обработан.
⏭️ Пропускаем чанк 2, он уже обработан.
⏭️ Пропускаем чанк 3, он уже обработан.
⏭️ Пропускаем чанк 4, он уже обработан.
⏭️ Пропускаем чанк 5, он уже обработан.


In [28]:
!pip install rich  
from rich.console import Console
from rich.markup import escape
import difflib

console = Console()

# Функция для подсветки исправлений
def highlight_changes(original, corrected):
    diff = difflib.ndiff(original.split(), corrected.split())
    highlighted_text = []

    for word in diff:
        if word.startswith("- "):  # Удалённое слово (ошибка)
            highlighted_text.append(f"[red]{escape(word[2:])}[/red]")
        elif word.startswith("+ "):  # Добавленное слово (исправление)
            highlighted_text.append(f"[green]{escape(word[2:])}[/green]")
        else:
            highlighted_text.append(word[2:])

    return " ".join(highlighted_text)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [29]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

# Указываем последний чекпоинт
checkpoint_path = "./t5-spell-corrector/checkpoint-8436"  # Укажи реальный номер

# Загружаем токенизатор из исходной модели, а не из чекпоинта
original_model_name = "UrukHan/t5-russian-spell"  # Убедись, что это та же модель
tokenizer = AutoTokenizer.from_pretrained(original_model_name)

# Загружаем модель из чекпоинта
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_path).to("cuda" if torch.cuda.is_available() else "cpu")

print(f"✅ Модель загружена из {checkpoint_path}")
print(f"✅ Токенизатор загружен из {original_model_name}")



✅ Модель загружена из ./t5-spell-corrector/checkpoint-8436
✅ Токенизатор загружен из UrukHan/t5-russian-spell


In [30]:
def correct_text(input_text):
    """
    Функция принимает текст с ошибками, передаёт его в модель и возвращает исправленный текст.
    """
    # Формируем запрос к модели
    input_text = "Исправь текст: " + input_text
    inputs = tokenizer(input_text, return_tensors="pt", max_length=512, truncation=True).to(model.device)

    # Генерация исправленного текста
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)

    # Декодируем результат
    corrected_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return corrected_text


In [35]:
# Пример текста с ошибками
test_texts = ["сеглдыя хорош ден",  # сегодня хороший день
    "я нвидел табя",      # я видел тебя
    "превет как дила",    # привет как дела
    "девочка ишла до магизина"  # девочка шла в магазин
]

# Проверяем, как модель исправляет текст
for text in test_texts:
    corrected = correct_text(text)
    print(f"❌ Оригинал: {text}")
    print(f"✅ Исправлено: {corrected}\n")


❌ Оригинал: сеглдыя хорош ден
✅ Исправлено: «Сегодня хорош день.

❌ Оригинал: я нвидел табя
✅ Исправлено: Я не видел тебя.

❌ Оригинал: превет как дила
✅ Исправлено: «Ответ как дила»

❌ Оригинал: девочка ишла до магизина
✅ Исправлено: Девочка ишла до магизина.

